In [18]:
import numpy as np
import rasterio as rs
import matplotlib.pyplot as plt
from numpy import loadtxt
from osgeo import gdal, gdalconst
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [19]:
#Plot Image
def read_data_EMP(inras):
    # Read data
    img = gdal.Open(inras, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    img_rgbnir = img[:,:,[3,2,1,7]]
    return img_rgbnir
def MyNormalize(img_i,sigma):
        nr,nc,nb = img_i.shape
        img_n = np.zeros(shape=(nr,nc,nb))
        for i in range(0,nb):
            one_band = img_i[:,:,i]
            mi = np.min(one_band)
            ma = np.max(one_band)
            one_band = (one_band-mi)/(ma-mi+np.finfo(float).eps)
            img_n[:,:,i] = sigma*one_band
        return img_n 

In [20]:
#Import Imagery
asisub = 'Input/Imagery/Subset_SingleDateImage_Amsterdam.tif'
amisub = 'Input/Imagery/Subset_MedianImage_Amsterdam.tif'
msisub = 'Input/Imagery/Subset_SingleDateImage_Milano.tif'
mmisub = 'Input/Imagery/Subset_MedianImage_Milano.tif'
bsisub = 'Input/Imagery/Subset_SingleDateImage_Budapest.tif'
bmisub = 'Input/Imagery/Subset_MedianImage_Budapest.tif'
#Import Feature extraction
EMPas = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empasi.npy')
NDWIas = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwiasi.csv', delimiter=',')
NDVIas = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndviasi.csv', delimiter=',')
EMPam = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empami.npy')
NDWIam = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwiami.csv', delimiter=',')
NDVIam = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndviami.csv', delimiter=',')
EMPms = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empmsi.npy')
NDWIms = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwimsi.csv', delimiter=',')
NDVIms = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndvimsi.csv', delimiter=',')
EMPmm = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empmmi.npy')
NDWImm = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwimmi.csv', delimiter=',')
NDVImm = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndvimmi.csv', delimiter=',')
EMPbs = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empbsi.npy')
NDWIbs = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwibsi.csv', delimiter=',')
NDVIbs = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndvibsi.csv', delimiter=',')
EMPbm = np.load('D:/Thesis/Notebooks/Output/Sub/EMP/empbmi.npy')
NDWIbm = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDWI/ndwibmi.csv', delimiter=',')
NDVIbm = loadtxt('D:/Thesis/Notebooks/Output/Sub/NDVI/ndvibmi.csv', delimiter=',')
#Import Twitter
Twita = rs.open('Input/Twitter/Subset_TwitterRaster_Amsterdam.tif','r').read()
Twita = np.transpose(Twita, [1, 2, 0])
Twita = Twita[:,:,0]
Twitm = rs.open('Input/Twitter/Subset_TwitterRaster_Milano.tif','r').read()
Twitm = np.transpose(Twitm, [1, 2, 0])
Twitm = Twitm[:,:,0]
Twitb = rs.open('Input/Twitter/Subset_TwitterRaster_Budapest.tif','r').read()
Twitb = np.transpose(Twitb, [1, 2, 0])
Twitb = Twitb[:,:,0]
#Import Validation datasets
Vala = rs.open('Input/Validation/Validation_30m_Amsterdam_sub.tif','r').read()
Vala = np.transpose(Vala, [1, 2, 0])
Val30ma = Vala[:,:,0]
Val30ma = np.where(Val30ma==2, 1, 0)
Val30ma = Val30ma.flatten()
Val2a = rs.open('Input/Validation/Validation_GAIA_Amsterdam_sub.tif','r').read()
Val2a = np.transpose(Val2a, [1, 2, 0])
Val2GAIAa = Val2a[:,:,0]
Val2GAIAa = np.where(Val2GAIAa>0, 1, 0)
Val2GAIAa = Val2GAIAa.flatten()
Val3a = rs.open('Input/Validation/Validation_GHS_Amsterdam_sub.tif', 'r').read()
Val3a = np.transpose(Val3a, [1, 2, 0])
Val3GHSa = Val3a[:,:,0]
Val3GHS50a = np.where(Val3GHSa>50, 1, 0)
Val3GHS50a = Val3GHS50a.flatten()

Valm = rs.open('Input/Validation/Validation_30m_Milano_sub.tif','r').read()
Valm = np.transpose(Valm, [1, 2, 0])
Val30mm = Valm[:,:,0]
Val30mm = np.where(Val30mm==2, 1, 0)
Val30mm = Val30mm.flatten()
Val2m = rs.open('Input/Validation/Validation_GAIA_Milano_sub.tif','r').read()
Val2m = np.transpose(Val2m, [1, 2, 0])
Val2GAIAm = Val2m[:,:,0]
Val2GAIAm = np.where(Val2GAIAm>0, 1, 0)
Val2GAIAm = Val2GAIAm.flatten()
Val3m = rs.open('Input/Validation/Validation_GHS_Milano_sub.tif', 'r').read()
Val3m = np.transpose(Val3m, [1, 2, 0])
Val3GHSm = Val3m[:,:,0]
Val3GHS50m = np.where(Val3GHSm>50, 1, 0)
Val3GHS50m = Val3GHS50m.flatten()

Valb = rs.open('Input/Validation/Validation_30m_Budapest_sub.tif','r').read()
Valb = np.transpose(Valb, [1, 2, 0])
Val30mb = Valb[:,:,0]
Val30mb = np.where(Val30mb==2, 1, 0)
Val30mb = Val30mb.flatten()
Val2b = rs.open('Input/Validation/Validation_GAIA_Budapest_sub.tif','r').read()
Val2b = np.transpose(Val2b, [1, 2, 0])
Val2GAIAb = Val2b[:,:,0]
Val2GAIAb = np.where(Val2GAIAb>0, 1, 0)
Val2GAIAb = Val2GAIAb.flatten()
Val3b = rs.open('Input/Validation/Validation_GHS_Budapest_sub.tif', 'r').read()
Val3b = np.transpose(Val3b, [1, 2, 0])
Val3GHSb = Val3b[:,:,0]
Val3GHS50b = np.where(Val3GHSb>50, 1, 0)
Val3GHS50b = Val3GHS50b.flatten()

from numpy import *
GLCM_ASI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM/ASI/GLCM_ASI_sub.csv', delimiter=',')
img = read_data_EMP(asisub)
nr, nc, nb = img.shape
nf = 16
GLCMas = GLCM_ASI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMas)
GLCMas[where_are_NaNs] = 0

GLCM_AMI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM/AMI/GLCM_AMI_sub.csv', delimiter=',')
img = read_data_EMP(amisub)
nr, nc, nb = img.shape
nf = 16
GLCMam = GLCM_AMI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMam)
GLCMam[where_are_NaNs] = 0

GLCM_MSI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM/MSI/GLCM_MSI_sub.csv', delimiter=',')
img = read_data_EMP(msisub)
nr, nc, nb = img.shape
nf = 16
GLCMms = GLCM_MSI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMms)
GLCMms[where_are_NaNs] = 0

GLCM_MMI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM/MMI/GLCM_MMI_sub.csv', delimiter=',')
img = read_data_EMP(mmisub)
nr, nc, nb = img.shape
nf = 16
GLCMmm = GLCM_MMI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMmm)
GLCMmm[where_are_NaNs] = 0

GLCM_BSI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM/BSI/GLCM_BSI_sub.csv', delimiter=',')
img = read_data_EMP(bsisub)
nr, nc, nb = img.shape
nf = 16
GLCMbs = GLCM_BSI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMbs)
GLCMbs[where_are_NaNs] = 0

GLCM_BMI_load = loadtxt('D:/Thesis/Notebooks/Output/Sub/GLCM//BMI/GLCM_BMI_sub.csv', delimiter=',')
img = read_data_EMP(bmisub)
nr, nc, nb = img.shape
nf = 16
GLCMbm = GLCM_BMI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMbm)
GLCMbm[where_are_NaNs] = 0

## IF asi sub

In [21]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(asisub)
        stack = np.concatenate((img, EMPas, GLCMas), axis=2)
        stacked = np.dstack((stack, NDVIas, NDWIas))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twita
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30ma
        true2 = Val2GAIAa
        true3 = Val3GHS50a
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 80.50.6917568046995536
Optimal P parameter 50.70.7209486381747721
Optimal R parameter 100.30.9465143904528839
Optimal F parameter 120.40.6689946795855839
Optimal AUC parameter 80.50.7415860527112277
Wall time: 1min 33s


## IF ami sub

In [22]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(amisub)
        stack = np.concatenate((img, EMPam, GLCMam), axis=2)
        stacked = np.dstack((stack, NDVIam, NDWIam))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twita
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30ma
        true2 = Val2GAIAa
        true3 = Val3GHS50a
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 110.60.6984810606998177
Optimal P parameter 90.70.7177627564192837
Optimal R parameter 100.30.966003920609563
Optimal F parameter 70.50.6726246624743727
Optimal AUC parameter 110.60.7493622346016253
Wall time: 1min 34s


## IF msi sub

In [23]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(msisub)
        stack = np.concatenate((img, EMPms, GLCMms), axis=2)
        stacked = np.dstack((stack, NDVIms, NDWIms))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitm
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mm
        true2 = Val2GAIAm
        true3 = Val3GHS50m
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 60.40.7089428479493082
Optimal P parameter 60.70.7635349359874785
Optimal R parameter 50.30.9835913827517272
Optimal F parameter 50.30.7716623401023982
Optimal AUC parameter 60.70.7443357658894989
Wall time: 1min 35s


## IF mmi sub

In [24]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(mmisub)
        stack = np.concatenate((img, EMPmm, GLCMmm), axis=2)
        stacked = np.dstack((stack, NDVImm, NDWImm))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitm
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mm
        true2 = Val2GAIAm
        true3 = Val3GHS50m
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 110.30.7117233071788652
Optimal P parameter 120.70.7586016286469287
Optimal R parameter 80.30.9810309253431536
Optimal F parameter 110.30.7733597754610605
Optimal AUC parameter 70.70.735367636007235
Wall time: 1min 30s


## IF bsi sub

In [25]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(bsisub)
        stack = np.concatenate((img, EMPbs, GLCMbs), axis=2)
        stacked = np.dstack((stack, NDVIbs, NDWIbs))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitb
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mb
        true2 = Val2GAIAb
        true3 = Val3GHS50b
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 90.60.7550720691732052
Optimal P parameter 60.70.7704216266932504
Optimal R parameter 110.30.9789812976517099
Optimal F parameter 50.60.7397899560505575
Optimal AUC parameter 90.60.7874621370780197
Wall time: 1min 27s


## IF bmi sub

In [26]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(bmisub)
        stack = np.concatenate((img, EMPbm, GLCMbm), axis=2)
        stacked = np.dstack((stack, NDVIbm, NDWIbm))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitb
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mb
        true2 = Val2GAIAb
        true3 = Val3GHS50b
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + str(max_value4))

Optimal OA parameter 50.60.7591313746359414
Optimal P parameter 80.70.7929817767163665
Optimal R parameter 60.30.967972046899637
Optimal F parameter 70.50.7440820939834762
Optimal AUC parameter 50.60.7914763375678356
Wall time: 1min 31s


## SVM asi sub

In [33]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(asisub)
            stack = np.concatenate((img, EMPas, GLCMas), axis=2)
            stacked = np.dstack((stack, NDVIas, NDWIas))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twita
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30ma
            true2 = Val2GAIAa
            true3 = Val3GHS50a
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.50.4_0.6865754147999636
Optimal P parameter rbf0.70.7_0.7142109737754234
Optimal R parameter linear0.20.3_0.9996409466038322
Optimal F parameter rbf0.40.3_0.6667870119162295
Optimal AUC parameter rbf0.50.4_0.7335126853205396
Wall time: 9.92 s


## SVM ami sub

In [34]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(amisub)
            stack = np.concatenate((img, EMPam, GLCMam), axis=2)
            stacked = np.dstack((stack, NDVIam, NDWIam))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twita
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30ma
            true2 = Val2GAIAa
            true3 = Val3GHS50a
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.40.5_0.6934976857008489
Optimal P parameter rbf0.40.7_0.7180445680836883
Optimal R parameter linear0.80.3_1.0
Optimal F parameter rbf0.30.4_0.6653806525934035
Optimal AUC parameter rbf0.40.5_0.7408354295516318
Wall time: 10.5 s


## SVM msi sub

In [35]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(msisub)
            stack = np.concatenate((img, EMPms, GLCMms), axis=2)
            stacked = np.dstack((stack, NDVIms, NDWIms))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twitm
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30mm
            true2 = Val2GAIAm
            true3 = Val3GHS50m
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.30.3_0.7099824261775699
Optimal P parameter rbf0.80.7_0.7733238315814478
Optimal R parameter linear0.30.3_0.9954783362030489
Optimal F parameter rbf0.30.3_0.7663062583470244
Optimal AUC parameter rbf0.40.5_0.7481506234838319
Wall time: 14.7 s


## SVM mmi sub

In [36]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(mmisub)
            stack = np.concatenate((img, EMPmm, GLCMmm), axis=2)
            stacked = np.dstack((stack, NDVImm, NDWImm))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twitm
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30mm
            true2 = Val2GAIAm
            true3 = Val3GHS50m
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.20.4_0.7168056896281444
Optimal P parameter rbf0.80.7_0.7692093302461195
Optimal R parameter linear0.70.3_0.999079899521881
Optimal F parameter rbf0.20.3_0.7707072120605912
Optimal AUC parameter rbf0.50.4_0.7494396926147445
Wall time: 15.4 s


## SVM bsi sub

In [37]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(bsisub)
            stack = np.concatenate((img, EMPbs, GLCMbs), axis=2)
            stacked = np.dstack((stack, NDVIbs, NDWIbs))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twitb
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30mb
            true2 = Val2GAIAb
            true3 = Val3GHS50b
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.20.5_0.7304687177710122
Optimal P parameter rbf0.80.7_0.78658857798023
Optimal R parameter linear0.80.3_0.9951753799405391
Optimal F parameter rbf0.30.3_0.7193181386103445
Optimal AUC parameter rbf0.20.5_0.7590148622825885
Wall time: 13.7 s


## SVM bmi sub

In [38]:
%%time
#IF
kernel = ["rbf", "linear"]
nu = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for k in kernel:
    for n in nu:
        for t in threshold:
            x = str(k)+str(n)+str(t)
            img = read_data_EMP(bmisub)
            stack = np.concatenate((img, EMPbm, GLCMbm), axis=2)
            stacked = np.dstack((stack, NDVIbm, NDWIbm))
            #Setup for IF and OCSVM
            X = stacked
            Ytr1 = Twitb
            nr,nc,nb = X.shape
            ns = nr*nc
            X = X.reshape((ns,nb))
            Ytr = Ytr1.reshape((ns,))
            ind = np.where(Ytr > 0)
            Xtr = X[ind[0],:]
            Ytr = Ytr[ind[0]]
            standard_scaler = StandardScaler()
            Xtr = standard_scaler.fit_transform(Xtr) 
            X = standard_scaler.transform(X)
            model = svm.OneClassSVM(nu=n, kernel=k)
            model.fit(Xtr)
            yhat = model.score_samples(X)
            yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
            class_map1 = np.reshape(yhat1,(nr,nc))
            class_map = np.where(class_map1>t, 1, 0)
            class_map = class_map.flatten()
            pred = class_map
            true1 = Val30mb
            true2 = Val2GAIAb
            true3 = Val3GHS50b
            cfm1 = confusion_matrix(true1, pred)
            TN1,FP1,FN1,TP1= cfm1.ravel()
            cfm2 = confusion_matrix(true2, pred)
            TN2,FP2,FN2,TP2= cfm2.ravel()
            cfm3 = confusion_matrix(true3, pred)
            TN3,FP3,FN3,TP3= cfm3.ravel()

            AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
                  ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
                  ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
            AOscores[x] = AO
        
            P = ((TP1/(TP1+FP1))+
                 (TP2/(TP2+FP2))+
                 (TP3/(TP3+FP3)))/3
            Pscores[x] = P
        
            R = ((TP1/(TP1+FN1))+
                 (TP2/(TP2+FN2))+
                 (TP3/(TP3+FN3)))/3
            Rscores[x] = R
        
            F = (((2*TP1)/(2*TP1+FP1+FN1))+
                 ((2*TP2)/(2*TP2+FP2+FN2))+
                 ((2*TP3)/(2*TP3+FP3+FN3)))/3
            Fscores[x] = F
        
            auc1 = roc_auc_score(true1,pred)
            auc2 = roc_auc_score(true2,pred)
            auc3 = roc_auc_score(true3,pred)
            score = ((auc1+auc2+auc3)/3)
            AUCscores[x] = score
        
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('Optimal OA parameter ' + str(max_key) + '_' + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('Optimal P parameter ' + str(max_key1) + '_' + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('Optimal R parameter ' + str(max_key2) + '_' + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('Optimal F parameter ' + str(max_key3) + '_' + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('Optimal AUC parameter ' + str(max_key4) + '_' + str(max_value4))

Optimal OA parameter rbf0.70.4_0.7386945867676543
Optimal P parameter rbf0.80.7_0.8179849054049481
Optimal R parameter linear0.40.3_0.986907265787022
Optimal F parameter rbf0.30.3_0.7198412283090714
Optimal AUC parameter rbf0.80.3_0.7683536262292471
Wall time: 14.6 s
